In [1]:
import sys
import os


notebook_dir = os.path.dirname(os.path.realpath("__file__"))  # Path to notebooks directory
parent_dir = os.path.dirname(notebook_dir)  # Parent directory of notebooks

sys.path.append(parent_dir)

In [2]:
import numpy as np
import matplotlib.pyplot as plt

from rc.risk_control import rc_main

In [5]:
PATH = '/data/'
DATASET = "imagenet"
MODELS = ['msdnet', 'dvit', 'l2wden',  'dynperc']

LAMBDAS = np.arange(0., 1.001, 0.01)[::-1]  # Descending order
DELTA = 0.1
EPSILONS = np.arange(0.01, 0.51, 0.01)
N_CAL = 100
N_TRIALS = 50
RCP_TYPES = ['ucb-wsr', 'crc']
RISK_TYPES = ['prediction-gt-gap', 'confidence-brier', 'prediction-consistency', 'confidence-brier-top-pred']

In [ ]:
PLOT_RES = {}
for model in MODELS:
    # Iterate over each risk_type and populate the grid
    for row, risk_type in enumerate(RISK_TYPES):
        print(f"Model: {model}, n_cal: {N_CAL}, risk_type: {risk_type}")
        losses, exits = img_class_inference(model=model, dataset=DATASET, 
                                            path=PATH, lambdas=LAMBDAS, 
                                            risk_type=risk_type, non_negative=False)
        if 'brier' in risk_type:
            B = 2.
        else:
            B = 1.
        test_risk, eff_gains, _ = rc_main(losses, exits, eps_grid=EPSILONS, 
                                        rcp_types=RCP_TYPES, 
                                        delta=DELTA, n_cal=N_CAL, n_trials=N_TRIALS, loss_bound=B)
        PLOT_RES[(model, risk_type)] = (test_risk, eff_gains)        